## Products summary ##

Pandas DataFrame **purchases** stores information about purchases: **customer**, **product**, **amount**. DataFrame **products** stores products' prices. DataFrame **discounts** stores discounts (in %) for customers.
We want to conduct a function, that would take as an input information on these three DataFrames. It would return  DataFrame with rows as customers from purchases, columns as all products, and values as the total sum for particular customer and product.

In [2]:
import pandas as pd
import numpy as np

In [3]:
def totals(purchases, products, discounts):
    result = {}
    for customer_name in set(purchases['customer']):
        customers_purchases = purchases.loc[purchases['customer'] == customer_name]
        for item in customers_purchases['item']:
            result.setdefault(item, {})
            amount = customers_purchases.loc[customers_purchases['item'] == item]['quantity'].values[0]
            price = products.loc[products['product'] == item]['price'].values[0]
            customer_with_discount = discounts.loc[discounts['customer'] == customer_name] 
            discount = customer_with_discount['discount'].values[0] if not customer_with_discount.empty else 0
            result[item][customer_name] = result[item].setdefault(customer_name, 0) + price * amount * (1 - discount / 100)

    for product in products['product']:
        result.setdefault(product, dict.fromkeys(list(set(purchases['customer'].values)), 0.0))
        for customer_name in set(purchases['customer']):
            result[product].setdefault(customer_name, 0.0)
    
    products = pd.Index(list(result.keys()), name='product')
    customers = pd.Index(list(result[products[0]].keys()), name='customer')
    
    data = []
    for name in customers.values:
        data.append([])
        for product in products.values:
            data[-1].append(result[product][name])

    return pd.DataFrame(data,index=customers, columns=products)

In [4]:
# Example

purchases = pd.DataFrame([['Ivan', 'sweeties', 4],
                          ['Petr', 'chocolate', 5],
                          ['Ivan', 'chocolate', 3],
                          ['Anna', 'juice', 2]],
                        columns=['customer', 'item', 'quantity'])
products = pd.DataFrame([['sweeties', 15],
                      ['chocolate', 7],
                      ['juice', 8],
                      ['lemons', 3]], columns=['product', 'price'])
discounts = pd.DataFrame([['Ivan', 10],
                         ['Petr', 5],
                         ['Vasily', 15]], 
                         columns=['customer', 'discount'])

totals(purchases, products, discounts)

product,sweeties,chocolate,juice,lemons
customer,,,,
Ivan,54.0,18.90,0.0,0.0
Anna,0.0,0.00,16.0,0.0
Petr,0.0,33.25,0.0,0.0
